Проверим, корректно ли ICU сортирует? Правда ли тут https://stackoverflow.com/a/1098160 написана?

Зависимости под Windows:

```
> pip install pipwin
> pipwin install pyicu
```

Увы, не всегда работает. Поэтому ставим PyICU вручную отсюда: https://www.lfd.uci.edu/~gohlke/pythonlibs/

* В роли Windows выступала Windows 10 x86\_64
* В роли UNIX-подобной ОС выступал Linux x86\_64 5.10.16-arch1-1

Для Mac OS X:

Ничего невозможного, [но постараться придётся](https://gist.github.com/ddelange/6e04e81b99fae08e817a00515d4a378d).

In [1]:
import os

strings = ['a','b','c','"b"','"c"','ä', 'cote', 'coté', 'côte', 'côté', 'и', 'й', 'і', 'ї']
locales_win = ['en_US', 'fr_FR', 'fr_CA', 'ru_RU']
locales_unix = [l + '.UTF-8' for l in locales_win]

locales = locales_win if os.name == 'nt' else locales_unix

Следующий пример под Windows считает любой французский канадским, под Linux работает верно, под Mac OS X — если вы знаете, как настроить несколько локалей

In [2]:
import locale
import functools
import os
# https://docs.python.org/3/library/functools.html#functools.cmp_to_key

ol = locale.getlocale()

for l in locales:
    locale.setlocale(locale.LC_COLLATE, l)
    print(l, sorted(strings, key=functools.cmp_to_key(locale.strcoll)))

locale.setlocale(locale.LC_COLLATE, ol[0][:2] if os.name == 'nt' else '.'.join(ol))

en_US.UTF-8 ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'й', 'і', 'ї']
fr_FR.UTF-8 ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'й', 'і', 'ї']
fr_CA.UTF-8 ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'й', 'і', 'ї']
ru_RU.UTF-8 ['"b"', '"c"', 'a', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'ä', 'и', 'й', 'і', 'ї']


'ru_RU.UTF-8'

Для Linux все локали придётся сгенерировать при помощи `locale-gen`

Следующий пример будет правильно работать под Windows и UNIX (Linux, OS X)

In [3]:
import icu

for l in locales_win:  # locales_win в том числе и под UNIX, иначе канадский французский не сработает...
    collator  = icu.Collator.createInstance(icu.Locale(l))
    print(l, sorted(strings, key=collator.getSortKey))

en_US ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'і', 'ї', 'й']
fr_FR ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'і', 'ї', 'й']
fr_CA ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'côte', 'coté', 'côté', 'и', 'і', 'ї', 'й']
ru_RU ['"b"', '"c"', 'и', 'і', 'ї', 'й', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté']
